In [2]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Lambda
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

In [6]:
def create_input():
    X = []
    y = []
    df = pd.read_csv("classification_data/attack_img/attack_data.csv")
    shuffled_df = df.sample(len(df))

    print(shuffled_df.head)
    s = 0
    for entry in shuffled_df.values:
        s += 1
        # print(s)
        entry_img = cv2.imread(f"classification_data/attack_img/{entry[0]}")

        prepared_y = np.delete(entry,0)

        X.append(entry_img)
        y.append(prepared_y)

    
    X = np.asarray(X)
    y = np.asarray(y).astype(np.float32)
    return X, y


In [9]:
def compile_model():
    image_width, image_height, channels = 640, 360, 3  # Assuming RGB images

    model = Sequential()
    model.add(Input(shape=(image_height, image_width, channels)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='linear'))

    model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

    return model

model = compile_model()

In [10]:
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])


In [7]:
X, y = create_input()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)

<bound method NDFrame.head of            filename    x    y
267  attack_267.jpg  346   45
8      attack_8.jpg  382  104
265  attack_265.jpg  408   64
201  attack_201.jpg  562  122
61    attack_61.jpg  378   86
..              ...  ...  ...
79    attack_79.jpg  345   54
184  attack_184.jpg  415  244
270  attack_270.jpg  318  197
217  attack_217.jpg  375   84
7      attack_7.jpg  383   39

[325 rows x 3 columns]>
(260, 360, 640, 3) (260, 2)


In [15]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.8148 - loss: 28142030.0000 - val_accuracy: 0.0000e+00 - val_loss: 326316.0312
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.4173 - loss: 274543.5000 - val_accuracy: 1.0000 - val_loss: 3953.7593
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.9303 - loss: 70538.8828 - val_accuracy: 1.0000 - val_loss: 33430.2773
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.9842 - loss: 39565.0156 - val_accuracy: 0.0000e+00 - val_loss: 28981.1328
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.7069 - loss: 21217.7676 - val_accuracy: 1.0000 - val_loss: 11851.2568
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.9979 - loss: 7824.6597 - val_accuracy: 1.0000 - val_loss: 3741.9453
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.9954 - loss: 4822.7090 - val_accuracy: 1.0000 - val_loss: 3201.0923
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.9965 - 

In [16]:
loss, accuracy = model.evaluate(X_val, y_val)
print('Test accuracy:', accuracy)

model.save('mouse_attack.h5')

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step - accuracy: 1.0000 - loss: 2679.9746


Test accuracy: 1.0


In [16]:
model_rap = tf.keras.models.load_model('mouse_attack.h5')


for n in range(5):
    x_real, y_real = int(y_val[n][0]), int(y_val[n][1])
    print(x_real, y_real)
    img = np.asarray(X_val[n])
    
    tbp = np.expand_dims(img, axis=0)
    predictions = model_rap.predict(tbp)
    x, y = int(predictions[0][0]), int(predictions[0][1]),
    print(x, y, "\n",x_real, y_real)
    cv2.circle(img, (x,y), 10, (0,0,255), -1)
    cv2.circle(img, (x_real, y_real), 10, (255,0,0), -1)
    cv2.imshow('ss', img)
    cv2.waitKey(0) & 0xFF
cv2.destroyWindow('ss')

373 104
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
347 130 
 373 104
365 101
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
323 108 
 365 101
352 165
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
396 184 
 352 165
297 212
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
399 154 
 297 212
401 138
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
383 144 
 401 138


In [28]:
model_rap = tf.keras.models.load_model('my_mobilenet_model.h5')
model_rap.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_rap.fit(X_train, y_train, epochs=40, validation_data=(X_val, y_val))
model_rap.save('class_2.h5')

Epoch 1/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.7854 - loss: 0.5956 - val_accuracy: 0.7400 - val_loss: 0.7340
Epoch 2/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.8160 - loss: 0.5927 - val_accuracy: 0.7000 - val_loss: 0.7350
Epoch 3/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7564 - loss: 0.6643 - val_accuracy: 0.6900 - val_loss: 0.7348
Epoch 4/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7663 - loss: 0.5880 - val_accuracy: 0.7200 - val_loss: 0.7174
Epoch 5/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7939 - loss: 0.5774 - val_accuracy: 0.7500 - val_loss: 0.6913
Epoch 6/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7758 - loss: 0.5911 - val_accuracy: 0.7700 - val_loss: 0.6796
Epoch 7/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.8515 - loss: 0.5431 - val_accuracy: 0.7200 - val_loss: 0.6956
Epoch 8/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.8156 - loss: 0.5335 - val_accuracy: 0.7600 - val_loss: